In [1]:
VERSION = '99_v1_03_03'

In [2]:
model_list = [    
    {'path':'../../13_Baseline5/exp/result/13_v1_03/oof_df.csv', 'w':1}, #deberta-v3-base, seed100, split00_v2_07
    #{'path':'../../13_Baseline5/exp/result/13_v1_04/oof_df.csv', 'w':1}, #deberta-v3-base, seed200, split00_v2_07
    {'path':'../../13_Baseline5/exp/result/13_v1_07/oof_df.csv', 'w':1}, #deberta-base, seed100, split00_v2_07
    
    {'path':'../../18_BaselineSW/exp/result/18_v2_01/oof_df.csv', 'w':1}, #deberta-large, seed100, split00_v2_07
    {'path':'../../18_BaselineSW/exp/result/18_v2_03/oof_df.csv', 'w':1}, #deberta-v3-large, seed100, split00_v2_07
    {'path':'../../18_BaselineSW/exp/result/18_v2_04/oof_df.csv', 'w':1}, #longformer-large, seed100, split00_v2_07
    #{'path':'../../18_BaselineSW/exp/result/18_v2_07/oof_df.csv', 'w':1}, #deberta-large, seed200, split00_v2_07
    #{'path':'../../18_BaselineSW/exp/result/18_v1_04/oof_df.csv', 'w':1}, #deberta-v3-large, seed200, split00_v2_07
    {'path':'../../18_BaselineSW/exp/result/18_v1_05/oof_df.csv', 'w':1}, #deberta-xlarge, seed100, split00_v2_07
    {'path':'../../18_BaselineSW/exp/result/18_v2_22/oof_df.csv', 'w':1}, #deberta-v2-xlarge, seed100, split00_v2_07
    
    #{'path':'../../19_RNN/exp/result/19_v1_01/oof_df.csv', 'w':1}, #deberta-large, seed100, split00_v2_07, lstm
    
    {'path':'../../20_WoSpanDet/exp/result/20_v1_01/oof_df.csv', 'w':1}, #deberta-large, seed100, split00_v2_07, huggingface pre
    #{'path':'../../20_WoSpanDet/exp/result/20_v1_02/oof_df.csv', 'w':1}, #deberta-v3-large, seed100, split00_v2_07, huggingface pre
    
    {'path':'../../22_BaselineMLM/exp/result/22_v1_01/oof_df.csv', 'w':1}, #deberta-large, seed100, split00_v2_07, MLM pre
]

In [3]:
import pandas as pd

w_list = []
for i,model in enumerate(model_list):
    w = model['w']
    w_list.append(w)

stack_df = None
for i,model in enumerate(model_list):
    pred_df = pd.read_csv(model['path'])
    w = w_list[i]
    pred_df['pred_ineffective'] = pred_df['pred_ineffective'] * w / sum(w_list)
    pred_df['pred_adequate'] = pred_df['pred_adequate'] * w / sum(w_list)
    pred_df['pred_effective'] = pred_df['pred_effective'] * w / sum(w_list)
    pred_df = pred_df.rename(columns={
        'pred_ineffective':f'Ineffective_{i}', 
        'pred_adequate':f'Adequate_{i}', 
        'pred_effective':f'Effective_{i}'
    })
    if stack_df is None:
        stack_df = pred_df.copy()
    else:
        stack_df = stack_df.merge(pred_df[['discourse_id',f'Ineffective_{i}',f'Adequate_{i}',f'Effective_{i}']],
                                  on='discourse_id', how='left')
    

train_df = pd.read_csv('../../input/feedback-prize-effectiveness/train.csv')
oof_df = train_df[['discourse_id']].merge(stack_df, on='discourse_id', how='left')

In [4]:
for col_name in ['Ineffective','Adequate','Effective']:
    cols = [f'{col_name}_{i}' for i in range(len(model_list))]
    oof_df[col_name] = oof_df[cols].sum(axis=1)

In [5]:
from sklearn.metrics import log_loss

score = log_loss(oof_df['label'].values, oof_df[['Ineffective','Adequate','Effective']].values)
print('oof={:.4f}'.format(score))

oof=0.5824


In [6]:
import os
os.makedirs('result', exist_ok=True)

oof_df.to_csv(f'result/stack_{VERSION}.csv', index=False)

In [7]:
oof_df.head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,Ineffective_0,Adequate_0,Effective_0,label,loss,...,Effective_6,Ineffective_7,Adequate_7,Effective_7,Ineffective_8,Adequate_8,Effective_8,Ineffective,Adequate,Effective
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,0.003575,0.099885,0.007651,1,0.106507,...,0.002561,0.008685,0.099868,0.002558,0.009077,0.100483,0.001550,0.042171,0.903465,0.054364
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate,0.011417,0.098603,0.001091,1,0.119425,...,0.004341,0.005775,0.103320,0.002017,0.017741,0.091025,0.002344,0.086873,0.887982,0.025145
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate,0.030340,0.080298,0.000473,1,0.324790,...,0.003669,0.019661,0.090601,0.000848,0.032400,0.077171,0.001540,0.176633,0.807579,0.015788
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate,0.023196,0.086686,0.001229,1,0.248233,...,0.001464,0.021785,0.088375,0.000951,0.037760,0.072434,0.000917,0.231264,0.756292,0.012444
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate,0.034982,0.075694,0.000436,1,0.383832,...,0.001308,0.036201,0.074207,0.000704,0.047198,0.063289,0.000625,0.311455,0.679543,0.009001
